In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas_read_xml as pdx
from pandas_read_xml import auto_separate_tables

In [3]:
df_2 = pdx.read_xml('export.xml', transpose=True)

In [4]:
df_2

,HealthData
@locale,en_US
ExportDate,{'@value': '2020-08-18 21:01:15 -0500'}
Me,{'@HKCharacteristicTypeIdentifierDateOfBirth':...
Record,"[{'@type': 'HKQuantityTypeIdentifierHeight', '..."


In [5]:
data = df_2.pipe(auto_separate_tables, key_columns)

NameError: name 'key_columns' is not defined

In [6]:
#https://pypi.org/project/pandas-read-xml/

In [ ]:




columns = ["name", "email", "age"]

datatframe = pd.DataFrame(columns = columns)

for node in root: 

    name = node.attrib.get("name")

    mail = node.find("email").text if node is not None else None

    age = node.find("age").text if node is not None else None

    datatframe = datatframe.append(pd.Series([name, mail, age], index = columns), ignore_index = True)

In [7]:
import xml.etree.ElementTree as etree

In [9]:
tree = etree.parse("export.xml")

In [16]:
root = tree.getroot()

In [27]:
root.tag, root.attrib

('HealthData', {'locale': 'en_US'})

In [24]:
columns = ['creationDate', 'startDate', 'endDate', 'value']

In [25]:
df = pd.DataFrame(columns = columns)

In [26]:
for node in root: 

    creationDate = node.attrib.get("creationDate")

    startDate = node.find("startDate").text if node is not None else None

    endDate = node.find("endDate").text if node is not None else None
    
    value = node.find("value").text if node is not None else None

    df = datatframe.append(pd.Series([creationDate, startDate, endDate, value], index = columns), ignore_index = True)

AttributeError: 'NoneType' object has no attribute 'text'